In [1]:
import io, os
import codecs, re, urlparse, urllib2
import time
from PIL import Image as PI
import pyocr
import pyocr.builders
from wand.image import Image

In [2]:
# List of PDF URLs from Colombian website
colombia_filenames = 'contract_data/Contracts_Backlog/colombia_oil_contracts.txt' 
# Where you want to save the PDFspip 
destination_folder = 'contract_data/Contracts_Backlog/'

In [3]:
existing_pdfs = [unicode(f,'utf-8') for f in os.listdir(destination_folder) if f.lower().endswith('.pdf')]
f = codecs.open(colombia_filenames,'r', encoding='utf-8').readlines()
print len(existing_pdfs)

474


In [4]:
def urlEncodeNonAscii(b):
    return re.sub('[\x80-\xFF]', lambda c: '%%%02x' % ord(c.group(0)), b)

def iriToUri(iri):
    parts= urlparse.urlparse(iri)
    return urlparse.urlunparse(
        part.encode('idna') if parti==1 else urlEncodeNonAscii(part.encode('utf-8'))
        for parti, part in enumerate(parts)
    )

In [5]:
def download_pdf(url,pdf_filename):
    '''
    Function downloads PDFs from url to given filename
    '''
    try:
        url = iriToUri(url)
    except:
        pass
    response = urllib2.urlopen(url)
    f = open(destination_folder + pdf_filename, 'w')
    f.write(response.read())
    f.close()
    
    return

### Download PDFs

In [14]:
existing_pdfs = [f for f in os.listdir(destination_folder) if f.lower().endswith('.pdf')]

In [6]:
testurl = u'http://www.anh.gov.co/Asignacion-de-areas/Contratos%20EP%20y%20TEAS%20firmados/CTO%20EyP%20CAÑO%20LOS%20TOTUMOS%202010.pdf'
print testurl

http://www.anh.gov.co/Asignacion-de-areas/Contratos%20EP%20y%20TEAS%20firmados/CTO%20EyP%20CAÑO%20LOS%20TOTUMOS%202010.pdf


In [15]:
pdf_filename = testurl.strip().split('/')[-1].replace("%20","_")
print pdf_filename
print type(pdf_filename)
pdf_filename

CTO_EyP_CAÑO_LOS_TOTUMOS_2010.pdf
<type 'unicode'>


u'CTO_EyP_CA\xd1O_LOS_TOTUMOS_2010.pdf'

In [16]:
query = [item for item in existing_pdfs if 'TOTUMOS' in item][0]
print query
print type(query)
query

CTO_EyP_CAÑO_LOS_TOTUMOS_2010.pdf
<type 'str'>


'CTO_EyP_CAN\xcc\x83O_LOS_TOTUMOS_2010.pdf'

In [17]:
print query
print pdf_filename
query == pdf_filename

CTO_EyP_CAÑO_LOS_TOTUMOS_2010.pdf
CTO_EyP_CAÑO_LOS_TOTUMOS_2010.pdf


/Users/jaclynweiser/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  This is separate from the ipykernel package so we can avoid doing imports until


False

In [ ]:
for url in f:
    pdf_filename = url.strip().split('/')[-1].replace("%20","_").encode('utf8')
    if not pdf_filename in existing_pdfs:
        print pdf_filename
        try:
            download_pdf(url.strip(),pdf_filename)
        except:
            print "Sleeping on " + pdf_filename
            time.sleep(5*60)
            try:
                download_pdf(url.strip(),pdf_filename)
            except:
                print "Could not open file: " + pdf_filename

### Perform OCR on PDFs

In [ ]:
def ocr_pdf_to_text(filename):
    tool = pyocr.get_available_tools()[0]
    lang = 'spa'
    req_image = []
    final_text = []
    image_pdf = Image(filename=filename, resolution=300)
    image_jpeg = image_pdf.convert('jpeg')
    for img in image_jpeg.sequence:
        img_page = Image(image=img)
        req_image.append(img_page.make_blob('jpeg'))

    for img in req_image: 
        txt = tool.image_to_string(
            PI.open(io.BytesIO(img)),
            lang=lang,
            builder=pyocr.builders.TextBuilder()
        )
        final_text.append(txt)
    return final_text

In [ ]:
colombian_backlog = []
for filename in filenames:
    colombian_backlog.append(ocr_pdf_to_text(filename))